# Zero-Coupon Bond Using Marlowe Runtime\'s REST API

***Before running this notebook, you might want to use Jupyter's "clear output" function to erase the results of the previous execution of this notebook. That will make more apparent what has been executed in the current session.***

The zero-coupon bond example is a simple Marlowe contract where a lender provides principal to a borrower who repays it back with interest.

[A video works through this Jupyter notebook.](https://youtu.be/wgJVdkM2pBY)

You can ask questions about Marlowe in [the #ask-marlowe channel on the IOG Discord](https://discord.com/channels/826816523368005654/936295815926927390) or post problems with this lesson to [the issues list for the Marlowe Starter Kit github repository](https://github.com/input-output-hk/marlowe-starter-kit/issues).

In this demonsration we use Marlowe Runtime\'s REST API, served via `marlowe-web-server`, to run this contract on Cardano\'s `preprod` public testnet. Marlowe contracts may use either addresses or role tokens for authorization: here we use role tokens and we have Marlowe Runtime mint them.

In [Marlowe Playground](https://play.marlowe.iohk.io//), the contract looks like this in Blockly format.

![Zero-coupon bond Marlowe contract](../../images/02-zcb-contract.png)

In Marlowe format it appears as
```
When
    [Case
        (Deposit
            (Role "Lender")
            (Role "Lender")
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
        )
        (Pay
            (Role "Lender")
            (Party (Role "Borrower"))
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
            (When
                [Case
                    (Deposit
                        (Role "Borrower")
                        (Role "Borrower")
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                    )
                    (Pay
                        (Role "Borrower")
                        (Party (Role "Lender"))
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                        Close 
                    )]
                (TimeParam "$BORROWER_DEADLINE")
                Close 
            )
        )]
    (TimeParam "$LENDER_DEADLINE")
    Close
```

## Preliminaries

See [Preliminaries](../../docs/preliminaries.md) for information on setting up one's environment for using this tutorial.

The first step is to check we have all the required tools and environment variables available to the notebook. 

In [ ]:
export SCRIPTS=../../scripts
export KEYS=../../keys
source $SCRIPTS/check-tools-and-env.sh

Make sure you've also [setup and funded](../../setup/01-setup-keys.ipynb) the different parties

- Lender
    - \$KEYS/lender.address: Cardano address for the lender
    - \$KEYS/lender.skey: location of signing key file for the lender
- Borrower
    - \$KEYS/borrower.address: Cardano address for the borrower
    - \$KEYS/borrower.skey: location of signing key file for the borrower

### Lender address and funds

Check that an address and key has been created for the lender. If not, see "Creating Addresses and Signing Keys" in [setup/01-setup-keys.ipynb](../../setup/01-setup-keys.ipynb).

In [ ]:
LENDER_SKEY=$KEYS/lender.skey
LENDER_ADDR=$(cat $KEYS/lender.address)
echo "LENDER_ADDR = $LENDER_ADDR"

Check that the lender has at least one hundred ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
$SCRIPTS/cardano-scan-address.sh $LENDER_ADDR

### Borrower address and funds

Check that an address and key has been created for the borrower.

In [ ]:
BORROWER_SKEY=$KEYS/borrower.skey
BORROWER_ADDR=$(cat $KEYS/borrower.address)
echo "BORROWER_ADDR = $BORROWER_ADDR"

Check that the borrower has at least one hundred ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
$SCRIPTS/cardano-scan-address.sh $BORROWER_ADDR

## Design the contract

The zero-coupon bond contract can be downloaded from the [Marlowe Playground](https://play.marlowe.iohk.io//) as a JSON file, or it can be generated using [Marlowe CLI](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli#readme) using the `marlowe-cli template` command.

Set the loan's principal to 80 ada and it's interest to 5 ada.

In [ ]:
ADA=1000000  # 1 ada = 1,000,000 lovelace
PRINCIPAL=$((80 * ADA))
INTEREST=$((5 * ADA))
echo "PRINCIPAL = $PRINCIPAL lovelace"
echo "INTEREST = $INTEREST lovelace"

On the Cardano blockchain, the protocol parameters require that each UTxO contain at least some ada. Here we will start the contract with 2 ada.

In [ ]:
MIN_LOVELACE="$((2 * ADA))"
echo "MIN_LOVELACE = $MIN_LOVELACE lovelace"

Later in the example we will need some constants for converting times.

In [ ]:
SECOND=1000 # 1 second = 1000 milliseconds
MINUTE=$((60 * SECOND)) # 1 minute = 60 seconds
HOUR=$((60 * MINUTE)) # 1 hour = 60 minutes

### *Alternative 1:* Use Marlowe Playground to design the contract

If you want to create the contract in Marlowe Playground, do the following:

1. Visit https://play.marlowe.iohk.io/ in a web browser.
2. Select "Open an Example".
3. Select "Marlowe" or "Blockly" under "Zero Coupon Bond".
4. Select "Send to Simulator".
5. Set the "Loan Deadline" to one hour into the future.
6. Set the "Payback Deadline" to three hours into the future.
7. Set the "Principal" to 80 ada.
8. Set the "Interest" to 5 ada.
9. Select "Download as JSON", set the file name to "zcb-contract.json", and store the file in this folder, namely [marlowe-starter-kit/lessons/02-runtime-rest/](.). Note that you can use Jupyter notebooks upload button

![Setting parameters for the zero-coupon bond contract in Marlowe Playground](../../images/zcb-playground.png)

### *Alternative 2:* Use Marlowe CLI to generate the contract

Below we generate the contract using Marlowe CLI.

First find the current time, measured in [POSIX milliseconds](https://en.wikipedia.org/wiki/Unix_time).

In [ ]:
NOW="$((`date -u +%s` * SECOND))"
echo NOW = "$NOW" POSIX milliseconds = "`date -d @$((NOW / SECOND))`"

The contract has a lending deadline and a repayment deadline. For convenience in this example, set the deadlines to the near future.

In [ ]:
LENDER_DEADLINE="$((NOW + 1 * HOUR))"
BORROWER_DEADLINE="$((NOW + 3 * HOUR))"
echo LENDER_DEADLINE = "$LENDER_DEADLINE" POSIX milliseconds = "`date -d @$((LENDER_DEADLINE / SECOND))`"
echo BORROWER_DEADLINE = "$BORROWER_DEADLINE" POSIX milliseconds = "`date -d @$((BORROWER_DEADLINE / SECOND))`"

Now create the JSON file for the contract, `zcb-contract.json`.

In [ ]:
marlowe-cli template zcb \
  --minimum-ada "$MIN_LOVELACE" \
  --lender Lender \
  --borrower Borrower \
  --principal "$PRINCIPAL" \
  --interest "$INTEREST" \
  --lending-deadline "$LENDER_DEADLINE" \
  --repayment-deadline "$BORROWER_DEADLINE" \
  --out-contract-file zcb-contract.json \
  --out-state-file /dev/null

The various command-line options are described by the help system.

In [ ]:
marlowe-cli template zcb --help

## Examine the contract

View the contract file as YAML.

In [ ]:
json2yaml zcb-contract.json

### \[Optional, but recommended\] Check the safety of the contract

If we were running the contract on the Cardano `mainnet`, then we\'d want to check its safety before creating it, so that there is no chance that we might lose funds.

Here are the steps for checking the safety of a contract:

1. Understand the [Marlowe Language](https://marlowe.iohk.io/).
2. Understand Cardano\'s [Extended UTxO Model](https://docs.cardano.org/learn/eutxo-explainer).
3. Read and understand the [Marlowe Best Practices Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Read and understand the [Marlowe Security Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Use [Marlowe Playground](https://play.marlowe.iohk.io//) to flag warnings, perform static analysis, and simulate the contract.
6. Use [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` tool to study whether the contract can run on a Cardano network.
7. Run *all execution paths* of the contract on a [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

See [Lesson 1](01-runtime-cli.ipynb) for an example of performing step 6.

## Transaction 1. Create the Contract

A `HTTP` `POST` request to Marlowe Runtime\'s `/contracts` endpoint will build the creation transaction for a Marlowe contract. We provide it the JSON file containing the contract and tell it the `MIN_LOVELACE` value that we previously chose. Anyone could create the contract, but in this example the lender will be doing so, so we provide their address to fund the transaction and to receive the change from it.

First we create the JSON body of the request to build the creation transaction.

In [ ]:
yaml2json << EOI > request-1.json
version: v1
contract: `cat zcb-contract.json`
roles:
  Lender: "$LENDER_ADDR"
  Borrower: "$BORROWER_ADDR"
minUTxODeposit: $MIN_LOVELACE
metadata: {}
tags: {}
EOI
cat request-1.json | jq

Next we post the request and view the response.

In [ ]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $LENDER_ADDR" \
  -d @request-1.json \
  -o response-1.json \
  -sS
json2yaml response-1.json

The identifier for the contract is embedded in the response.

In [ ]:
CONTRACT_ID="$(jq -r '.resource.contractId' response-1.json)"
echo "CONTRACT_ID = $CONTRACT_ID"

The CBOR serialization (in text-envelope format) is also embedded in the response.

In [ ]:
jq '.resource.txBody' response-1.json > tx-1.unsigned
du -hs tx-1.unsigned

There are many ways to sign and submit Cardano transactions:
- `cardano-cli` at the command line
- `cardano-wallet` at the command line or as a REST service
- `cardano-hw-cli` for a hardware wallet at the command line
- a Babbage-compatible CIP-30 wallet in a web browser
- `marlowe-cli` at the command line

For convenience, here we use `marlowe-cli transaction submit`. One may have to wait a minute or so for the transactions to be confirmed on the blockchain.

In [ ]:
echo "submiting transaction..."
TX_1=$(
marlowe-cli transaction submit \
  --tx-body-file tx-1.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"

One can view the transaction on the Cardano and Marlowe explorer and see that the contract has been created and the parties have received their role tokens. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "Cardano Scan (low level)"
$SCRIPTS/cardano-scan-tx.sh $TX_1
echo
echo "Marlowe Scan (high level)"
$SCRIPTS/marlowe-scan.sh $CONTRACT_ID

In particular, we see that the Marlowe contract holds the 2 ada that was set as `MINIMUM_LOVELACE`.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$CONTRACT_ID"

One can see that the lender and borrower have received their role tokens. Note that `4c656e646572 = Lender` and `426f72726f776572 = Borrower` in hexadecimal notation.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

## View the details of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}` can fetch a contract from the blockchain and return information about it.

In [ ]:
CONTRACT_URL="$MARLOWE_RT_WEBSERVER_URL/`jq -r '.links.contract' response-1.json`"
echo "CONTRACT_URL = $CONTRACT_URL"

In [ ]:
curl -sS "$CONTRACT_URL" | json2yaml

## Transaction 2. The lender deposits the principal

The lender deposits their 80 ada of principal into the contract using Marlowe Runtime\'s `HTTP` `POST` `/contract/{contractId}/transactions` endpoint. The lender is providing the funding for and receiving the change from this transaction, so we provide their address.

The deposit is represented as JSON input to the contract. The `marlowe-cli input deposit` tool conveniently formats the correct JSON for a deposit.

In [ ]:
marlowe-cli input deposit --help

In [ ]:
marlowe-cli input deposit \
  --deposit-party Lender \
  --deposit-account Lender \
  --deposit-amount "$PRINCIPAL" \
  --out-file input-2.json
json2yaml input-2.json

This input is included in the JSON request.

In [ ]:
yaml2json << EOI > request-2.json
version: v1
inputs: [$(cat input-2.json)]
metadata: {}
tags: {}
EOI
cat request-2.json | jq

Next we post the request and store the response.

In [ ]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $LENDER_ADDR" \
  -d @request-2.json \
  -o response-2.json \
  -sS
json2yaml response-2.json

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [ ]:
jq '.resource.txBody' response-2.json > tx-2.unsigned
du -hs tx-2.unsigned

In [ ]:
echo "submiting transaction..."
TX_2=$(
marlowe-cli transaction submit \
  --tx-body-file tx-2.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TX_2"

One can view the transaction on the Cardano and Marlowe explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "Cardano Scan (low level)"
$SCRIPTS/cardano-scan-tx.sh $TX_2
echo
echo "Marlowe Scan (high level)"
$SCRIPTS/marlowe-scan.sh $CONTRACT_ID

One can see that the lender has approximately 83 ada less than originally. Two ada were deposited in the contract when it was created and 80 ada were paid to the borrower in the second transaction; another 1 ada was attached to the role token that was sent to the borrower. The lender also holds their own role token.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

The Marlowe contract still has the 2 ada from its creation.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#1"

Marlowe\'s role-payout address holds the 80 ada on behalf of the borrower.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#3"

## View the further progress of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [ ]:
curl -sS "$CONTRACT_URL"/transactions/"$TX_2" | json2yaml

## Transaction 3. The borrower withdraws the principal

The principal of 80 ada is held at Marlowe's role-payout address for the benefit of the borrower. The borrower can withdraw these funds at any time. The contract ID and role name are included in the request body for a withdrawal.

In [ ]:
yaml2json << EOI > request-3.json
contractId: "$CONTRACT_ID"
role: Borrower
EOI
cat request-3.json

Next we post the request and store the response.

In [ ]:
curl "$MARLOWE_RT_WEBSERVER_URL/withdrawals" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $BORROWER_ADDR" \
  -d @request-3.json \
  -o response-3.json \
  -sS
json2yaml response-3.json

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [ ]:
jq '.resource.txBody' response-3.json > tx-3.unsigned
du -hs tx-3.unsigned

In [ ]:
echo "submiting transaction..."
TX_3=$(
marlowe-cli transaction submit \
  --tx-body-file tx-3.unsigned \
  --required-signer "$BORROWER_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_3 = $TX_3"

On can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
$SCRIPTS/cardano-scan-tx.sh $TX_3

The borrower now has about an additional 80 ada (the loan's principal). The borrower also holds their own role token.

In [ ]:
echo "Borrower:"
$SCRIPTS/cardano-scan-address.sh $BORROWER_ADDR
echo
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

## Transaction 4. The borrower repays the loan

After some time passes, the borrower repays principal plus interest. Thus, they fund the transaction and receive the change at their address.

First build the input to deposit the funds to the contract.

In [ ]:
marlowe-cli input deposit \
  --deposit-party Borrower \
  --deposit-account Borrower \
  --deposit-amount "$((PRINCIPAL+INTEREST))" \
  --out-file input-4.json
json2yaml input-4.json

Next build the request.

In [ ]:
yaml2json << EOI > request-4.json
version: v1
inputs: [$(cat input-4.json)]
metadata: {}
tags: {}
EOI
cat request-4.json | jq

Now post the request to Marlowe Runtime.

In [ ]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $BORROWER_ADDR" \
  -d @request-4.json \
  -o response-4.json \
  -sS
json2yaml response-4.json

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [ ]:
jq '.resource.txBody' response-4.json > tx-4.unsigned
du -hs tx-4.unsigned

In [ ]:
echo "submiting transaction..."
TX_4=$(
marlowe-cli transaction submit \
  --tx-body-file tx-4.unsigned \
  --required-signer "$BORROWER_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_4 = $TX_4"

One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "Cardano Scan (low level)"
$SCRIPTS/cardano-scan-tx.sh $TX_4
echo
echo "Marlowe Scan (high level)"
$SCRIPTS/marlowe-scan.sh $CONTRACT_ID

The borrower now has a total of about 5 ada (the loan's interest) less than originally.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

The 85 ada are at Marlowe\'s role-payout address, held for the benefit of the lender.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_4#2"

The Marlowe contract has closed, so there is no output to its script address.

## View the completion of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [ ]:
curl -sS "$CONTRACT_URL"/transactions/"$TX_4" | json2yaml

## Transaction 5. The lender withdraws their principal and interest

Here again we withdraw funds from the Marlowe role-payout address.

In [ ]:
yaml2json << EOI > request-5.json
contractId: "$CONTRACT_ID"
role: Lender
EOI
cat request-5.json

Next we post the request and store the response.

In [ ]:
curl "$MARLOWE_RT_WEBSERVER_URL/withdrawals" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $LENDER_ADDR" \
  -d @request-5.json \
  -o response-5.json \
  -sS
json2yaml response-5.json

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [ ]:
jq '.resource.txBody' response-5.json > tx-5.unsigned
du -hs tx-5.unsigned

In [ ]:
echo "submiting transaction..."
TX_5=$(
marlowe-cli transaction submit \
  --tx-body-file tx-5.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_5 = $TX_5"

One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
$SCRIPTS/cardano-scan-tx.sh $TX_5

The lender now has an additional 5 ada (the loan's interest), minus fees, compared to their original balance before the contract started.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

## *Appendix:* OpenAPI Description of Marlowe Web Services

Here is how to obtain the OpenAPI description of endpoints and types provided by `marlowe-web-server`.

In [ ]:
curl -sS "$MARLOWE_RT_WEBSERVER_URL/openapi.json" | json2yaml | head -n 20

It is also available at https://docs.marlowe.iohk.io/api/get-contracts.

![Marlowe Runtime OpenAPI](../../images/openapi.png)